#Power Plant ML Pipeline Application
This is an end-to-end example of using a number of different machine learning algorithms to solve a supervised regression problem.

###Table of Contents
- *Step 1: Business Understanding*
- *Step 2: Extract-Transform-Load (ETL) Your Data*
- *Step 3: Explore Your Data*
- *Step 4: Visualize Your Data*
- *Step 5: Data Preparation*
- *Step 6: Data Modeling*


*We are trying to predict power output given a set of readings from various sensors in a gas-fired power generation plant.  Power generation is a complex process, and understanding and predicting power output is an important element in managing a plant and its connection to the power grid.*

More information about Peaker or Peaking Power Plants can be found on Wikipedia https://en.wikipedia.org/wiki/Peaking_power_plant


Given this business problem, we need to translate it to a Machine Learning task.  The ML task is regression since the label (or target) we are trying to predict is numeric.


The example data is provided by UCI at [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)

You can read the background on the UCI page, but in summary we have collected a number of readings from sensors at a Gas Fired Power Plant

(also called a Peaker Plant) and now we want to use those sensor readings to predict how much power the plant will generate.


More information about Machine Learning with Spark can be found in the programming guide in the [SparkML Guide](https://spark.apache.org/docs/latest/mllib-guide.html)


*Please note this example only works with Spark version 1.4 or higher*

In [2]:
assert int(sc.version.replace(".", "")) >= 140, "Spark 1.4.0+ is required to run this notebook. Please attach it to a Spark 1.4.0+ cluster."

##Step 1: Business Understanding
The first step in any machine learning task is to understand the business need. 

As described in the overview we are trying to predict power output given a set of readings from various sensors in a gas-fired power generation plant.

The problem is a regression problem since the label (or target) we are trying to predict is numeric

##Step 2: Load (ETL) Your Data

Now that we understand what we are trying to do, the first step is to load our data into a format we can query and use.  This is known as ETL or "Extract-Transform-Load".  We will load our file from Amazon s3.

Note: Alternatively we could upload our data using "Databricks Menu > Tables > Create Table", assuming we had the raw files on our local computer.

Our data is available on Amazon s3 at the following path:  
`dbfs:/databricks-datasets/power-plant/data`

**ToDo:** Let's start by printing the first 5 lines of the file.  
*Hint*: To read the file into an RDD use `sc.textFile("dbfs:/databricks-datasets/power-plant/data")`  
*Hint*: Then you will need to figure out how to `take` and print the first 5 lines of the RDD.

In [5]:
rawTextRdd = sc.textFile("dbfs:/databricks-datasets/power-plant/data")
for line in rawTextRdd.take(5):
    print(line)


The file is a .tsv (Tab Seperated Values) file of floating point numbers.  

Our schema definition from UCI appears below:

- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output.  This is the value we are trying to predict given the measurements above.


**ToDo:** Transform the RDD so that each row is a tuple of float values.  Then print the first 5 rows.  
*Hint:* Use filter to exclude lines that start with AT to remove the header.  
*Hint:* Use map to transform each line into a PowerPlantRow of data fields.  
*Hint:* Use python's str.split break up each line into individual fields.

In [7]:
from collections import namedtuple
PowerPlantRow=namedtuple("PowerPlantRow", ["AT", "V", "AP", "RH", "PE"])
rawDataRdd=rawTextRdd\
  .map(lambda x: x.split("\t"))\
  .filter(lambda line: line[0] != "AT")\
  .map(lambda line: PowerPlantRow(float(line[0]), float(line[1]), float(line[2]), float(line[3]), float(line[4])))
rawDataRdd.take(5)

##Step 3: Explore Your Data
Now that your data is loaded, let's explore it, verify it, and do some basic analysis and visualizations.

**ToDo:** Transform your `rawDataRdd` into a Dataframe named `power_plant`.  Then use the `display(power_plant)` function to visualize it.

In [10]:
powerPlant=rawDataRdd.toDF()
display(powerPlant)

Next, let's register our dataframe as an SQL table.  Because this lab may be run many times, we'll take the precaution of removing any existing tables first.

**ToDo:** Execute the prepared code in the following cell...

In [12]:
sqlContext.sql("DROP TABLE IF EXISTS power_plant")
dbutils.fs.rm("dbfs:/user/hive/warehouse/power_plant", True)


**ToDo:** Register your `powerPlant` dataframe as the table named `power_plant`

In [14]:
sqlContext.registerDataFrameAsTable(powerPlant, "power_plant")

**ToDo:** Perform the query `SELECT * FROM power_plant`

In [16]:
%sql SELECT * FROM power_plant

**ToDo:** Use the `desc power_plant` SQL command to describe the schema

In [18]:
%sql desc power_plant

**Schema Definition**

Our schema definition from UCI appears below:

- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output

PE is our label or target. This is the value we are trying to predict given the measurements.

*Reference [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)*

**ToDo:** Display summary statistics for the the columns.  
*Hint:* To access the table from python use `sqlContext.table("power_plant")`  
*Hint:* We can use the describe function with no parameters to get some basic stats for each column like count, mean, max, min and standard deviation. The describe function is a method attached to a dataframe. More information can be found in the [Spark API docs](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.DataFrame)

In [21]:
display(sqlContext.table("power_plant").describe())

##Step 4: Visualize Your Data

To understand our data, we will look for correlations between features and the label.  This can be important when choosing a model.  E.g., if features and a label are linearly correlated, a linear model like Linear Regression can do well; if the relationship is very non-linear, more complex models such as Decision Trees can be better. We use Databrick's built in visualization to view each of our predictors in relation to the label column as a scatter plot to see the correlation between the predictors and the label.

**ToDo:** Do a scatter plot of Power(PE) as a function of Temperature (AT).  
*Bonus:* Name the y-axis "Power" and the x-axis "Temperature"

In [24]:
%sql select AT as Temperature, PE as Power from power_plant

Notice there appears to be a strong linear correlation between temperature and Power Output

**ToDo:** Do a scatter plot of Power(PE) as a function of ExhaustVacuum (V).  
*Bonus:* Name the y-axis "Power" and the x-axis "ExhaustVacuum"

In [27]:
%sql select V as ExhaustVacuum, PE as Power from power_plant;

The linear correlation is not as strong between Exhaust Vacuum Speed and Power Output but there is some semblance of a pattern.

**ToDo:** Do a scatter plot of Power(PE) as a function of Pressure (AP).  
*Bonus:* Name the y-axis "Power" and the x-axis "Pressure"

In [30]:
%sql select AP as Pressure, PE as Power from power_plant;

**ToDo:** Do a scatter plot of Power(PE) as a function of Humidity (RH).  
*Bonus:* Name the y-axis "Power" and the x-axis "Humidity"

In [32]:
%sql select RH Humidity, PE Power from power_plant;

...and atmospheric pressure and relative humidity seem to have little to no linear correlation

##Step 5: Data Preparation

The next step is to prepare the data. Since all of this data is numeric and consistent this is a simple task for us today.

We will need to convert the predictor features from columns to Feature Vectors using [pyspark.ml.feature.VectorAssembler](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler)

The VectorAssembler will be the first step in building our ML pipeline.

**ToDo:** Assign the data frame to the local variable `dataset`  
**ToDo:** Create a VectorAssembler named `vectorizer` to read your dataset.

In [35]:
from pyspark.ml.feature import VectorAssembler

dataset = sqlContext.table("power_plant")

vectorizer = VectorAssembler()
vectorizer.setInputCols(["AT", "V", "AP", "RH"])
vectorizer.setOutputCol("features")


##Step 6: Data Modeling
Now let's model our data to predict what the power output will be given a set of sensor readings

Our first model will be based on simple linear regression since we saw some linear patterns in our data based on the scatter plots during the exploration stage.

In machine learning we often will split up our initial data set into a "trainingSet" used to train our model and a "testSet" to evaluate the model's performance in giving predictions.

**ToDo:** Divide up the model into a trainingSet and a testSet.  Then cache each set into memory to maximize performance.   
**Hint:** Use the [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function.
**Hint:** The data set won't actually be cached until you perform an action on the dataset.  
*Tip*: For reproducability it's often best to use a predefined seed when doing the random sampling.

In [37]:
(trainingSet, testSet) = dataset.randomSplit((0.80, 0.20), seed=1800009193L)
(trainingSet.cache().count(), testSet.cache().count())

Next we'll create a Linear Regression Model and use the built in help to identify how to train it. See more details at [Linear Regression](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression) in the ML guide. 

**ToDo**: Run the next cell.

In [39]:
# ***** LINEAR REGRESSION MODEL ****

from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml import Pipeline

# Let's initialize our linear regression learner
lr = LinearRegression()

# We use explain params to dump the parameters we can use
print(lr.explainParams())

The cell below is based on the Spark ML pipeline API. More information can be found in the Spark ML Programming Guide at https://spark.apache.org/docs/latest/ml-guide.html

**ToDo:** Read, understand, and then run the next cell to train our linear regression model.

In [41]:
# Now we set the parameters for the method
lr.setPredictionCol("Predicted_PE")\
  .setLabelCol("PE")\
  .setMaxIter(100)\
  .setRegParam(0.1)

# We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
lrPipeline = Pipeline()
lrPipeline.setStages([vectorizer, lr])

# Let's first train on the entire dataset to see what we get
lrModel = lrPipeline.fit(trainingSet)

Now let's apply this model to the training set.

In [43]:
predictionsAndLabels = lrModel.transform(testSet)

display(predictionsAndLabels.select("AT", "V", "AP", "RH", "PE", "Predicted_PE"))

Since Linear Regression is Simply a Line of best fit over the data that minimizes the square of the error, given multiple input dimensions we can express each predictor as a line function of the form:

\\[ y = a + b x_1 + b x_2 + b x_i ... \\]

where a is the intercept and b are coefficients.

To express the coefficients of that line we can retrieve the Estimator stage from the PipelineModel and express the weights and the intercept for the function.

In [45]:
# The intercept is as follows:
intercept = lrModel.stages[1].intercept
print(intercept)


In [46]:
# The coefficents (i.e. weights) are as follows:

weights = lrModel.stages[1].weights.toArray()


featuresNoLabel = [col for col in dataset.columns if col != "PE"]

coefficents = sc.parallelize(weights).zip(sc.parallelize(featuresNoLabel))

# Now let's sort the coeffecients from the most to the least

equation = "y = {intercept}".format(intercept=intercept)
variables = []
for x in coefficents.sortByKey().collect():
    weight = abs(x[0])
    name = x[1]
    symbol = "+" if (x[0] > 0) else "-"
    equation += (" {} ({} * {})".format(symbol, weight, name))

# Finally here is our equation
print("Linear Regression Equation: " + equation)

Based on examining the output it shows there is a strong negative correlation between Atmospheric Temperature (AT) and Power Output.

But our other dimenensions seem to have little to no correlation with Power Output. Do you remember **Step 2: Explore Your Data**? When we visualized each predictor against Power Output using a Scatter Plot, only the temperature variable seemed to have a linear correlation with Power Output so our final equation seems logical.


Now let's see what our predictions look like given this model.

In [48]:
predictionsAndLabels = lrModel.transform(testSet)

display(predictionsAndLabels.select("AT", "V", "AP", "RH", "PE", "Predicted_PE"))

Now that we have real predictions we can use an evaluation metric such as Root Mean Squared Error (RMSE) to validate our regression model. RSME is defined as follows: \\( RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}\\)

The lower the Root Mean Squared Error, the better our model.

In [50]:
# Now let's compute some evaluation metrics against our test dataset

from pyspark.mllib.evaluation import RegressionMetrics

metrics = RegressionMetrics(predictionsAndLabels.select("Predicted_PE", "PE").rdd.map(lambda r: (float(r[0]), float(r[1]))))

rmse = metrics.rootMeanSquaredError
explainedVariance = metrics.explainedVariance
r2 = metrics.r2

print("Root Mean Squared Error: {}".format(rmse))
print("Explained Variance: {}".format(explainedVariance))
print("R2: {}".format(r2))

Generally a good model will have 68% of predictions within 1 RMSE and 95% within 2 RMSE of the actual value. Let's calculate and see if a RMSE of 4.51 meets this criteria.

In [52]:
# First we calculate the residual error and divide it by the RMSE
predictionsAndLabels.selectExpr("PE", "Predicted_PE", "PE - Predicted_PE Residual_Error", "(PE - Predicted_PE) / {} Within_RSME".format(rmse)).registerTempTable("Power_Plant_RMSE_Evaluation")

In [53]:
%sql SELECT * from Power_Plant_RMSE_Evaluation

In [54]:
%sql -- Now we can display the RMSE as a Histogram. Clearly this shows that the RMSE is centered around 0 with the vast majority of the error within 2 RMSEs.
SELECT Within_RSME  from Power_Plant_RMSE_Evaluation

We can see this definitively if we count the number of predictions within + or - 1.0 and + or - 2.0 and display this as a pie chart:

In [56]:
%sql SELECT case when Within_RSME <= 1.0 and Within_RSME >= -1.0 then 1  when  Within_RSME <= 2.0 and Within_RSME >= -2.0 then 2 else 3 end RSME_Multiple, COUNT(*) count  from Power_Plant_RMSE_Evaluation
group by case when Within_RSME <= 1.0 and Within_RSME >= -1.0 then 1  when  Within_RSME <= 2.0 and Within_RSME >= -2.0 then 2 else 3 end

So we have 68% of our training data within 1 RMSE and 97% (68% + 29%) within 2 RMSE. So the model is pretty decent. Let's see if we can tune the model to improve it further.